In [10]:
import numpy as np
import onnxruntime as ort
import onnx
from onnx.checker import check_model
from onnx.tools import update_model_dims
import glob
import pathlib
import warnings
import torch
import json
import pandas as pd
from tqdm import tqdm

In [11]:
def compute_mad(diff):
    diff = np.max(np.abs(diff))
    return float(np.max(diff))

def compute_max_l1(diff):
    diff = np.sum(np.abs(diff))
    return float(np.max(diff))

def compute_max_l2(diff):
    diff = np.sqrt(np.sum(np.square(diff)))
    return float(np.max(diff))

In [12]:
def filter_failed(path:str):
    root_path = pathlib.Path(path)
    return list(root_path.glob("*.onnx"))


def get_model(path: str):
    root_path = pathlib.Path(path)
    in_out_path = list(root_path.glob("*.npy"))
    model_path = list(root_path.glob("*.onnx"))

    in_outs = []
    for paths in in_out_path:
        with open(paths, 'rb') as f:
            in_out = np.load(f, allow_pickle=True)
            in_out = in_out.item()
        in_outs.append(in_out)
    if model_path:
        model = onnx.load_model(model_path[0])
        # print(in_out['input_shapes'])
        # model = update_model_dims.update_inputs_outputs_dims(
        #     model,
        #     in_out['input_shapes'],
        #     in_out['output_shapes']
        #     # {k: v.shape for k, v in self.torch_model.input_like.items()},
        #     # {k: v.shape for k, v in self.torch_model.output_like.items()},
        # )
    return in_outs, model, model_path[0]

In [13]:
def compute_distances(ort_outputs, outputs):
    matched_out = {key: {} for key in outputs}
    for key, val in outputs.items():
        diffs = []
        for out in ort_outputs:
            if out.shape == val.shape:
                diff = out.astype(float) - val.detach().numpy().astype(float)
                diffs.append(diff)
        # print(key, len(diffs))
        # matched_out[key]['l1'] = np.array(list(map(compute_max_l1, diffs)))
        # matched_out[key]['l2'] = np.array(list(map(compute_max_l2, diffs)))
        # matched_out[key]['mad'] = np.array(list(map(compute_mad, diffs)))
        # matched_out[key]['l1'] = list(map(compute_max_l1, diffs))
        # matched_out[key]['l2'] = list(map(compute_max_l2, diffs))
        matched_out[key]['mad'] = list(map(compute_mad, diffs))
    return matched_out

In [37]:
# onnx_models = list(
    # map(get_model, filter(filter_failed, glob.glob("./nnsmith_output/*")))
# )
onnx_models = map(get_model, filter(filter_failed, glob.glob("./nnsmith_output/*")))


In [38]:
ort.set_default_logger_severity(3)

# Catch models that throw exceptions at ORT

In [39]:
model_tests = {}
# errs = dict()
errs = set()
count = 0
for in_outs, model, model_path in tqdm(onnx_models):
    try:
        sess = ort.InferenceSession(onnx._serialize(model))
    except Exception as err:
        count += 1
        if str(err) in model_tests:
            model_tests[str(err)] += 1
        else:
            model_tests[str(err)] = 0
        errs = errs.union([str(err)])
        continue

1795it [03:35,  8.34it/s]


In [40]:
model_tests

{"[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for LeakyRelu(6) node with name '/mlist.17/LeakyRelu'": 4,
 "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Erf(13) node with name '/Erf_1'": 37,
 "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Resize(13) node with name '/Resize_4'": 12,
 "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Resize(13) node with name '/Resize_1'": 30,
 "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Resize(13) node with name '/Resize_2'": 16,
 "[ONNXRuntimeError] : 10 : INVALID_GRAPH : This is an invalid model. Type Error: Type 'tensor(int16)' of input parameter (onnx::Identity_17) of operator (ReduceMax) in node (/ReduceMax) is invalid.": 0,
 "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for Resize(13) node with name '/Resize_3'": 18,
 "[ONNXRuntimeError] : 9 : NOT_IMPL

In [41]:
invalid = 0
for i in model_tests:
    if "FAIL" in i:
        invalid += 1
invalid

0

In [42]:
len(errs)

59

In [43]:
count, errs

(396,
 {"[ONNXRuntimeError] : 10 : INVALID_GRAPH : This is an invalid model. Type Error: Type 'tensor(int16)' of input parameter (onnx::Identity_17) of operator (ReduceMax) in node (/ReduceMax) is invalid.",
  "[ONNXRuntimeError] : 10 : INVALID_GRAPH : This is an invalid model. Type Error: Type 'tensor(int8)' of input parameter (onnx::MatMul_28) of operator (MatMul) in node (/MatMul) is invalid.",
  "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for AveragePool(11) node with name '/mlist.10/AveragePool'",
  "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for AveragePool(11) node with name '/mlist.12/AveragePool'",
  "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for AveragePool(11) node with name '/mlist.13/AveragePool'",
  "[ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for AveragePool(11) node with name '/mlist.14/AveragePool'",
  "[ONNXRuntimeError] : 9 : NOT_IMPLEME

# Test Models

In [ ]:
onnx_models = map(get_model, filter(filter_failed, glob.glob("./nnsmith_output/*")))

In [ ]:

for in_outs, model, model_path in tqdm(onnx_models):
    model_tests[model_path.parts[-2]] = {}
    for i, in_out in enumerate(in_outs):
        inputs, outputs, in_shapes, out_shapes = in_out['inputs'], in_out['outputs'], in_out['input_shapes'], in_out['output_shapes']
        inputs = {key: val.numpy() for key, val in inputs.items()}
        try:
            sess = ort.InferenceSession(onnx._serialize(model))
        except Exception as err:
            continue
        # output_names = list(outputs.keys())
        output_names = [i.name for i in model.graph.output]
        ort_outputs = sess.run(output_names, inputs)
        
        matched_out = compute_distances(ort_outputs, outputs)
        # model_tests[model_path.parts[-2] + str(i)] = matched_out
        model_tests[model_path.parts[-2]][str(i)] = matched_out



In [ ]:
with open("pt_diff_test.json", 'w') as f:
    json.dump(model_tests, f)

# PT Diff Test Processing

In [43]:
with open("./pt_results_con/pt_diff_test.json", 'r') as f:
    matched_out = json.load(f)

In [44]:
len(matched_out)

1792

In [45]:
for key in matched_out:
    if matched_out[key]:
        for in_outs in matched_out[key]:
            for out in matched_out[key][in_outs]:
                for dist in matched_out[key][in_outs][out]:
                    matched_out[key][in_outs][out][dist] = np.array(matched_out[key][in_outs][out][dist]).min()
    else:
        matched_out[key] = None

In [46]:
for key in matched_out:
    if matched_out[key]:
        for in_outs in matched_out[key]:
            matched_out[key][in_outs] = pd.DataFrame(matched_out[key][in_outs]).max(axis=1).to_dict()

In [47]:
averaged_distances = {}
for key in matched_out:
    if matched_out[key]:
        min_dist = pd.DataFrame(matched_out[key]).min(axis=1).to_dict()
        max_dist = pd.DataFrame(matched_out[key]).max(axis=1).to_dict()
        mean_dist = pd.DataFrame(matched_out[key]).mean(axis=1).to_dict()
        std_dev = pd.DataFrame(matched_out[key]).std(axis=1).to_dict()
        med_dist = pd.DataFrame(matched_out[key]).median(axis=1).to_dict()
        averaged_distances[key] = {"min_dist": min_dist, "max_dist": max_dist, "mean_dist": mean_dist, "std_dev": std_dev, "med_dist": med_dist}
    else:
        averaged_distances[key] = None

In [48]:
with open("./pt_results_con/pt_diff_test_processed.json", 'w') as f:
    json.dump(averaged_distances, f)

# TF Diff Test Processing

In [109]:
with open("./tf_results_con/tf_diff_test_con.json", 'r') as f:
    matched_out = json.load(f)

In [110]:
len(matched_out)

1803

In [111]:
for key in matched_out:
    if matched_out[key]:
        for in_outs in matched_out[key]:
            for out in matched_out[key][in_outs]:
                for dist in matched_out[key][in_outs][out]:
                    array = np.array(matched_out[key][in_outs][out][dist])
                    if array.any():
                        # matched_out[key][in_outs][out][dist] = np.array(matched_out[key][in_outs][out][dist]).min()
                        matched_out[key][in_outs][out][dist] = array.min()
                    else:
                        matched_out[key][in_outs][out][dist] = None
                        
    else:
        matched_out[key] = None

In [112]:
for key in matched_out:
    if matched_out[key]:
        for in_outs in matched_out[key]:
            matched_out[key][in_outs] = pd.DataFrame(matched_out[key][in_outs]).max(axis=1).to_dict()

In [113]:
averaged_distances = {}
for key in matched_out:
    if matched_out[key]:
        min_dist = pd.DataFrame(matched_out[key]).min(axis=1).to_dict()
        max_dist = pd.DataFrame(matched_out[key]).max(axis=1).to_dict()
        mean_dist = pd.DataFrame(matched_out[key]).mean(axis=1).to_dict()
        std_dev = pd.DataFrame(matched_out[key]).std(axis=1).to_dict()
        med_dist = pd.DataFrame(matched_out[key]).median(axis=1).to_dict()
        averaged_distances[key] = {"min_dist": min_dist, "max_dist": max_dist, "mean_dist": mean_dist, "std_dev": std_dev, "med_dist": med_dist}
    else:
        averaged_distances[key] = None

In [114]:
with open("./tf_results_con/tf_diff_test_processed_con.json", 'w') as f:
    json.dump(averaged_distances, f)

# PT Diff Results

In [503]:
processed_pt = pd.read_json('./pt_results_con/pt_diff_test_processed.json')

In [504]:
processed_pt

,100_1025018750_pt,100_103297041_pt,100_1143776810_pt,100_1149472975_pt,100_1167864533_pt,100_1241640024_pt,100_1257191948_pt,100_1277174399_pt,100_1329716773_pt,100_1330982796_pt,...,95_838781746_pt,95_877692505_pt,95_878293399_pt,95_879433627_pt,95_914793679_pt,95_918795419_pt,95_93851359_pt,95_946159942_pt,95_962693378_pt,95_979628108_pt
min_dist,{'mad': 4.76837158203125e-07},{'mad': 111324.0},{'mad': 103.5470199584961},{'mad': 9.264992880528553},{'mad': 35.0116081237793},{'mad': 1.52587890625e-05},{'mad': 2.0},{'mad': 231.0},{'mad': 1.862645149230957e-09},{'mad': 0.001896070534712},...,{'mad': 2.9802322387695312e-08},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 0.241669178009033},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 34.623046875},{'mad': 10191.753366470337},{'mad': 5.0},{'mad': 22.081281661987305}
max_dist,{'mad': 1311918.0},{'mad': 1136129819.0},{'mad': 240.4823169708252},{'mad': 9.264992880528553},{'mad': 74.19530200958252},{'mad': 0.0001220703125},{'mad': 20.11418914794922},{'mad': 231.0},{'mad': 0.026824951171875003},{'mad': 23.0},...,{'mad': 0.0001220703125},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 10242.0},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 63.6689453125},{'mad': 17041.3076171875},{'mad': 5.0},{'mad': 28.478858947753906}
mean_dist,{'mad': 65536.35442084119},{'mad': 11963736.98},{'mad': 178.9645616722107},{'mad': 9.264992880528553},{'mad': 54.504786825180055},{'mad': 4.608154296875001e-05},{'mad': 12.668045024871827},{'mad': 231.0},{'mad': 0.000444759968668},{'mad': 3.630569855428912},...,{'mad': 1.6291439533233642e-05},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 2162.785990934372},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 45.39142578125},{'mad': 15122.531473824978},{'mad': 5.0},{'mad': 25.90479260444641}
std_dev,{'mad': 287088.32476524153},{'mad': 113553183.52901879},{'mad': 41.443319492569636},{'mad': 0.0},{'mad': 10.683811720296571},{'mad': 3.135233076921612e-05},{'mad': 3.498195515092353},{'mad': 0.0},{'mad': 0.0030524131347200003},{'mad': 4.26744354258174},...,{'mad': 2.5600133559010133e-05},{'mad': 0.0},{'mad': 0.0},{'mad': 2340.5709860461407},{'mad': 0.0},{'mad': 0.0},{'mad': 5.9697119054679355},{'mad': 1976.5180166540192},{'mad': 0.0},{'mad': 1.336387491018659}
med_dist,{'mad': 4.548871040344238},{'mad': 450715.5},{'mad': 182.15515089035034},{'mad': 9.264992880528553},{'mad': 52.969329833984375},{'mad': 3.0517578125e-05},{'mad': 13.027950286865234},{'mad': 231.0},{'mad': 2.86102294921875e-06},{'mad': 3.0},...,{'mad': 3.814697265625e-06},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 1553.0},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 45.0029296875},{'mad': 15972.424255371094},{'mad': 5.0},{'mad': 26.156956672668457}


In [505]:
processed_pt.dropna(axis=1, inplace=True)

In [482]:
x = processed_pt.transpose().applymap(lambda x: x['mad'],)

In [483]:
# processed_table = pd.DataFrame(columns=pd.MultiIndex.from_product([processed_pt.columns, ['l1', 'l2', 'mad',]]), index=processed_pt.index)
processed_table = pd.DataFrame(columns=pd.MultiIndex.from_product([processed_pt.columns,  ['mad',]]), index=processed_pt.index)

In [484]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = processed_pt.loc[label, col[0]][col[1]]

In [485]:
processed_table = processed_table.transpose()

In [486]:
processed_table

,,min_dist,max_dist,mean_dist,std_dev,med_dist
100_1025018750_pt,mad,0.0,1311918.0,65536.354421,287088.324765,4.548871
100_103297041_pt,mad,111324.0,1136129819.0,11963736.98,113553183.529019,450715.5
100_1143776810_pt,mad,103.54702,240.482317,178.964562,41.443319,182.155151
100_1149472975_pt,mad,9.264993,9.264993,9.264993,0.0,9.264993
100_1167864533_pt,mad,35.011608,74.195302,54.504787,10.683812,52.96933
...,...,...,...,...,...,...
95_918795419_pt,mad,0.821586,0.821586,0.821586,0.0,0.821586
95_93851359_pt,mad,34.623047,63.668945,45.391426,5.969712,45.00293
95_946159942_pt,mad,10191.753366,17041.307617,15122.531474,1976.518017,15972.424255
95_962693378_pt,mad,5.0,5.0,5.0,0.0,5.0


In [487]:
processed_table.loc[(processed_table == np.inf)['med_dist']]

,,min_dist,max_dist,mean_dist,std_dev,med_dist
100_713236053_pt,mad,0.0,inf,inf,None,inf
45_2063306003_pt,mad,inf,inf,inf,None,inf
60_787665175_pt,mad,inf,inf,inf,None,inf
70_1917615025_pt,mad,inf,inf,inf,None,inf
80_607027687_pt,mad,inf,inf,inf,None,inf
85_3153848255_pt,mad,inf,inf,inf,None,inf
90_3716620021_pt,mad,inf,inf,inf,None,inf
95_1028398433_pt,mad,inf,inf,inf,None,inf
95_789242032_pt,mad,inf,inf,inf,None,inf


In [488]:
processed_table = processed_table.loc[~(processed_table == np.inf)['med_dist']].reset_index(level=1)

In [489]:
x = processed_table.loc[processed_table['level_1'] == "mad"][['med_dist']]

In [490]:
processed_table.loc[processed_table['level_1'] == "mad"]

,level_1,min_dist,max_dist,mean_dist,std_dev,med_dist
100_1025018750_pt,mad,0.0,1311918.0,65536.354421,287088.324765,4.548871
100_103297041_pt,mad,111324.0,1136129819.0,11963736.98,113553183.529019,450715.5
100_1143776810_pt,mad,103.54702,240.482317,178.964562,41.443319,182.155151
100_1149472975_pt,mad,9.264993,9.264993,9.264993,0.0,9.264993
100_1167864533_pt,mad,35.011608,74.195302,54.504787,10.683812,52.96933
...,...,...,...,...,...,...
95_918795419_pt,mad,0.821586,0.821586,0.821586,0.0,0.821586
95_93851359_pt,mad,34.623047,63.668945,45.391426,5.969712,45.00293
95_946159942_pt,mad,10191.753366,17041.307617,15122.531474,1976.518017,15972.424255
95_962693378_pt,mad,5.0,5.0,5.0,0.0,5.0


In [311]:
x.loc[x['med_dist'] > 1e-3]

,med_dist
100_1025018750_pt,4.548871
100_103297041_pt,450715.5
100_1143776810_pt,182.155151
100_1149472975_pt,9.264993
100_1167864533_pt,52.96933
...,...
95_918795419_pt,0.821586
95_93851359_pt,45.00293
95_946159942_pt,15972.424255
95_962693378_pt,5.0


# TF Diff Results

In [507]:
processed_tf = pd.read_json('./tf_results_con/tf_diff_test_processed_con.json')

In [508]:
processed_tf

,100_100231791_tf,100_1110108348_tf,100_1313238182_tf,100_1334400342_tf,100_1345924174_tf,100_1446647357_tf,100_1468122792_tf,100_1474848908_tf,100_1476325540_tf,100_1580555511_tf,...,95_703149461_tf,95_714726297_tf,95_769140607_tf,95_830139270_tf,95_839227546_tf,95_875262176_tf,95_876568748_tf,95_889665252_tf,95_907233445_tf,95_980915527_tf
min_dist,NaN,{'mad': 8.940696716308594e-08},NaN,NaN,NaN,NaN,NaN,NaN,{'mad': 1.192092895507812e-06},NaN,...,{'mad': 0.0},NaN,{'mad': 2.86102294921875e-06},NaN,{'mad': 7.450580596923828e-08},{'mad': 2.9802322387695312e-08},NaN,NaN,NaN,{'mad': 0.0}
max_dist,NaN,{'mad': 1.341104507446289e-07},NaN,NaN,NaN,NaN,NaN,NaN,{'mad': 2.86102294921875e-06},NaN,...,{'mad': 2.384185791015625e-07},NaN,{'mad': 0.0029296875},NaN,{'mad': 0.000732421875},{'mad': 1.0},NaN,NaN,NaN,{'mad': 4.76837158203125e-07}
mean_dist,NaN,{'mad': 9.208917617797851e-08},NaN,NaN,NaN,NaN,NaN,NaN,{'mad': 1.493692398071289e-06},NaN,...,{'mad': 5.358830094337463e-08},NaN,{'mad': 9.28783416748047e-05},NaN,{'mad': 0.00017100423574400002},{'mad': 0.010000903159379002},NaN,NaN,NaN,{'mad': 3.337860107421875e-08}
std_dev,NaN,{'mad': 9.323831140703135e-09},NaN,NaN,NaN,NaN,NaN,NaN,{'mad': 3.939011690922129e-07},NaN,...,{'mad': 5.647377894905398e-08},NaN,{'mad': 0.000328701365822},NaN,{'mad': 0.000169887627975},{'mad': 0.099999908786078},NaN,NaN,NaN,{'mad': 1.0726828981438529e-07}
med_dist,NaN,{'mad': 8.940696716308594e-08},NaN,NaN,NaN,NaN,NaN,NaN,{'mad': 1.430511474609375e-06},NaN,...,{'mad': 3.725290298461914e-08},NaN,{'mad': 2.2888183593750003e-05},NaN,{'mad': 0.00021362304687500003},{'mad': 2.384185791015625e-07},NaN,NaN,NaN,{'mad': 0.0}


In [509]:
processed_tf.dropna(axis=1, inplace=True)

In [510]:
processed_pt

,100_1025018750_pt,100_103297041_pt,100_1143776810_pt,100_1149472975_pt,100_1167864533_pt,100_1241640024_pt,100_1257191948_pt,100_1277174399_pt,100_1329716773_pt,100_1330982796_pt,...,95_838781746_pt,95_877692505_pt,95_878293399_pt,95_879433627_pt,95_914793679_pt,95_918795419_pt,95_93851359_pt,95_946159942_pt,95_962693378_pt,95_979628108_pt
min_dist,{'mad': 4.76837158203125e-07},{'mad': 111324.0},{'mad': 103.5470199584961},{'mad': 9.264992880528553},{'mad': 35.0116081237793},{'mad': 1.52587890625e-05},{'mad': 2.0},{'mad': 231.0},{'mad': 1.862645149230957e-09},{'mad': 0.001896070534712},...,{'mad': 2.9802322387695312e-08},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 0.241669178009033},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 34.623046875},{'mad': 10191.753366470337},{'mad': 5.0},{'mad': 22.081281661987305}
max_dist,{'mad': 1311918.0},{'mad': 1136129819.0},{'mad': 240.4823169708252},{'mad': 9.264992880528553},{'mad': 74.19530200958252},{'mad': 0.0001220703125},{'mad': 20.11418914794922},{'mad': 231.0},{'mad': 0.026824951171875003},{'mad': 23.0},...,{'mad': 0.0001220703125},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 10242.0},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 63.6689453125},{'mad': 17041.3076171875},{'mad': 5.0},{'mad': 28.478858947753906}
mean_dist,{'mad': 65536.35442084119},{'mad': 11963736.98},{'mad': 178.9645616722107},{'mad': 9.264992880528553},{'mad': 54.504786825180055},{'mad': 4.608154296875001e-05},{'mad': 12.668045024871827},{'mad': 231.0},{'mad': 0.000444759968668},{'mad': 3.630569855428912},...,{'mad': 1.6291439533233642e-05},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 2162.785990934372},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 45.39142578125},{'mad': 15122.531473824978},{'mad': 5.0},{'mad': 25.90479260444641}
std_dev,{'mad': 287088.32476524153},{'mad': 113553183.52901879},{'mad': 41.443319492569636},{'mad': 0.0},{'mad': 10.683811720296571},{'mad': 3.135233076921612e-05},{'mad': 3.498195515092353},{'mad': 0.0},{'mad': 0.0030524131347200003},{'mad': 4.26744354258174},...,{'mad': 2.5600133559010133e-05},{'mad': 0.0},{'mad': 0.0},{'mad': 2340.5709860461407},{'mad': 0.0},{'mad': 0.0},{'mad': 5.9697119054679355},{'mad': 1976.5180166540192},{'mad': 0.0},{'mad': 1.336387491018659}
med_dist,{'mad': 4.548871040344238},{'mad': 450715.5},{'mad': 182.15515089035034},{'mad': 9.264992880528553},{'mad': 52.969329833984375},{'mad': 3.0517578125e-05},{'mad': 13.027950286865234},{'mad': 231.0},{'mad': 2.86102294921875e-06},{'mad': 3.0},...,{'mad': 3.814697265625e-06},{'mad': 2.1053466796875},{'mad': 112.74782371520996},{'mad': 1553.0},{'mad': 21.554325342178345},{'mad': 0.821585655212402},{'mad': 45.0029296875},{'mad': 15972.424255371094},{'mad': 5.0},{'mad': 26.156956672668457}


In [399]:
# processed_table_tf = pd.DataFrame(columns=pd.MultiIndex.from_product([processed_tf.columns, ['l1', 'l2', 'mad',]]), index=processed_tf.index)
processed_table_tf = pd.DataFrame(columns=pd.MultiIndex.from_product([processed_tf.columns, ['mad',]]), index=processed_tf.index)

In [400]:
processed_table_tf

,100_1110108348_tf,100_1476325540_tf,100_1693470318_tf,100_177588628_tf,100_1779945412_tf,100_1792079900_tf,100_1937211808_tf,100_2127785824_tf,100_2221625129_tf,100_2260473050_tf,...,95_4117815657_tf,95_4137703176_tf,95_544531693_tf,95_558640384_tf,95_66273377_tf,95_703149461_tf,95_769140607_tf,95_839227546_tf,95_875262176_tf,95_980915527_tf
,mad,mad,mad,mad,mad,mad,mad,mad,mad,mad,...,mad,mad,mad,mad,mad,mad,mad,mad,mad,mad
min_dist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max_dist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_dist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std_dev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
med_dist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [377]:
for label, row in processed_table_tf.iterrows():
    for col, cell in row.iteritems():
        processed_table_tf.loc[label, col] = processed_tf.loc[label, col[0]][col[1]]

In [378]:
processed_table_tf = processed_table_tf.transpose()
processed_table_tf

,,min_dist,max_dist,mean_dist,std_dev,med_dist
100_1110108348_tf,mad,0.0,0.0,0.0,0.0,0.0
100_1476325540_tf,mad,0.000001,0.000003,0.000001,0.0,0.000001
100_1693470318_tf,mad,0.0,0.000001,0.000001,0.0,0.0
100_177588628_tf,mad,0.0,0.0,0.0,0.0,0.0
100_1779945412_tf,mad,0.003906,0.007812,0.007617,0.000856,0.007812
...,...,...,...,...,...,...
95_703149461_tf,mad,0.0,0.0,0.0,0.0,0.0
95_769140607_tf,mad,0.000003,0.00293,0.000093,0.000329,0.000023
95_839227546_tf,mad,0.0,0.000732,0.000171,0.00017,0.000214
95_875262176_tf,mad,0.0,1.0,0.010001,0.1,0.0


In [379]:
processed_table_tf.loc[(processed_table_tf == np.inf)['mean_dist']].shape

(18, 5)

In [380]:
processed_table_tf = processed_table_tf.loc[~(processed_table_tf == np.inf)['mean_dist']].reset_index(level=1)

In [381]:
x = processed_table_tf.loc[processed_table_tf['level_1'] == "mad"][['med_dist']]

In [382]:
processed_table_tf.loc[processed_table_tf['level_1'] == "mad"]

,level_1,min_dist,max_dist,mean_dist,std_dev,med_dist
100_1110108348_tf,mad,0.0,0.0,0.0,0.0,0.0
100_1476325540_tf,mad,0.000001,0.000003,0.000001,0.0,0.000001
100_1693470318_tf,mad,0.0,0.000001,0.000001,0.0,0.0
100_177588628_tf,mad,0.0,0.0,0.0,0.0,0.0
100_1779945412_tf,mad,0.003906,0.007812,0.007617,0.000856,0.007812
...,...,...,...,...,...,...
95_703149461_tf,mad,0.0,0.0,0.0,0.0,0.0
95_769140607_tf,mad,0.000003,0.00293,0.000093,0.000329,0.000023
95_839227546_tf,mad,0.0,0.000732,0.000171,0.00017,0.000214
95_875262176_tf,mad,0.0,1.0,0.010001,0.1,0.0


In [383]:
x.loc[x['med_dist'] < 1e-3].shape

(754, 1)

# Parse Results PT

In [491]:
results_jsons = glob.glob("./pt_results_con/result_*.json")

In [492]:
results = []
for results_json in results_jsons:
    with open(results_json, "r") as f:
        lines = f.read()
    x = list(
        map(
            lambda x: json.loads(x),
            '\n{"name'.join(lines.split('{"name')).splitlines()[1:],
        )
    )
    results.extend(x)


In [493]:
results = pd.DataFrame(results)

In [494]:
results

,name,error,error_des,mad,ml1,ml2,time
0,nnsmith_output/75_3699546747_pt,0,{},0,0,0,0.170390
1,nnsmith_output/75_1415249710_pt,0,{},0,0,0,0.287998
2,nnsmith_output/75_3101108083_pt,0,{},0,0,0,0.198968
3,nnsmith_output/75_930342665_pt,0,{},0,0,0,0.213932
4,nnsmith_output/75_2692494188_pt,0,{},0,0,0,0.246698
...,...,...,...,...,...,...,...
1787,nnsmith_output/85_951328719_pt,0,{},0,0,0,0.258583
1788,nnsmith_output/85_3741262020_pt,0,{},0,0,0,0.288985
1789,nnsmith_output/85_1423103689_pt,0,{},0,0,0,0.314030
1790,nnsmith_output/85_2792832355_pt,0,{},0,0,0,0.172139


In [450]:
results['num_nodes'] = results['name'].apply(lambda x: int(x.split("/")[1].split("_")[0]))

In [451]:
results = results.sort_values('num_nodes',)

In [210]:
results['error'].value_counts()

0    1792
Name: error, dtype: int64

In [211]:
results['name'] = results['name'].apply(lambda x: x.split("/")[-1])

In [70]:
z = results.set_index("name").join(processed_pt.transpose())

In [71]:
z

,error,error_des,mad,ml1,ml2,time,num_nodes,min_dist,max_dist,mean_dist,std_dev,med_dist
name,,,,,,,,,,,,
15_3490005371_pt,0,{},0,0,0,0.045101,15,{'mad': 0.0},{'mad': 5.960464477539064e-08},{'mad': 7.748603820800783e-09},{'mad': 1.900034545372634e-08},{'mad': 0.0}
15_1830730569_pt,0,{},0,0,0,0.038187,15,{'mad': 77.9517822265625},{'mad': 77.9517822265625},{'mad': 77.9517822265625},{'mad': 0.0},{'mad': 77.9517822265625}
15_594356564_pt,0,{},0,0,0,0.028006,15,{'mad': 0.0},{'mad': 1.9073486328125e-06},{'mad': 2.527236938476562e-07},{'mad': 6.430854058756909e-07},{'mad': 0.0}
15_3868237654_pt,0,{},0,0,0,0.052564,15,{'mad': 4.76837158203125e-07},{'mad': 7.62939453125e-06},{'mad': 2.7370452880859368e-06},{'mad': 2.21034842847256e-06},{'mad': 1.9073486328125e-06}
15_3294824347_pt,0,{},0,0,0,0.051622,15,{'mad': 0.0},{'mad': 2.2888183593750003e-05},{'mad': 3.7074089050292966e-06},{'mad': 5.224190061294284e-06},{'mad': 1.192092895507812e-07}
...,...,...,...,...,...,...,...,...,...,...,...,...
100_2038814902_pt,0,{},0,0,0,0.302235,100,{'mad': 5.462057113647461},{'mad': 9.11375379562378},{'mad': 6.830924878120422},{'mad': 0.9517348178114311},{'mad': 6.753756046295166}
100_723451920_pt,0,{},0,0,0,0.332820,100,{'mad': 8.807302713394165},{'mad': 33024.0},{'mad': 1000.9915888047218},{'mad': 3568.263596337814},{'mad': 10.331679344177246}
100_2738858578_pt,0,{},0,0,0,0.346879,100,{'mad': 0.000244140625},{'mad': 0.03125},{'mad': 0.0032031250000000002},{'mad': 0.006504243640674001},{'mad': 0.0009765625}


In [72]:
z.to_json("./random_pt_con.json")

# Parse Results TF

In [511]:
results_jsons = glob.glob("./tf_results_con/result_*.json")

In [512]:
results = []
for results_json in results_jsons:
    with open(results_json, "r") as f:
        lines = f.read()
    x = list(
        map(
            lambda x: json.loads(x),
            '\n{"name'.join(lines.split('{"name')).splitlines()[1:],
        )
    )
    results.extend(x)


In [513]:
results = pd.DataFrame(results)

In [514]:
results

,name,error,error_des,mad,ml1,ml2,time
0,nnsmith_output/70_2930611783_tf,0,{},0,0,0,4.212481
1,nnsmith_output/70_2101016416_tf,0,{},0,0,0,4.455755
2,nnsmith_output/70_419470385_tf,0,{},0,0,0,4.386364
3,nnsmith_output/70_790115950_tf,0,{},0,0,0,4.270439
4,nnsmith_output/70_367034129_tf,0,{},0,0,0,4.456023
...,...,...,...,...,...,...,...
1795,nnsmith_output/20_3869654409_tf,0,{},0,0,0,3.836306
1796,nnsmith_output/20_172034505_tf,0,{},0,0,0,3.856664
1797,nnsmith_output/20_4085096594_tf,0,{},0,0,0,3.732279
1798,nnsmith_output/20_3688011142_tf,0,{},0,0,0,3.777499


In [515]:
results.loc[results['error'] == 1]

,name,error,error_des,mad,ml1,ml2,time


In [516]:
results['num_nodes'] = results['name'].apply(lambda x: int(x.split("/")[1].split("_")[0]))

In [517]:
results = results.sort_values('num_nodes',)

In [518]:
results['error'].value_counts()

0    1800
Name: error, dtype: int64

In [519]:
results['num_nodes'].value_counts()

70     101
15     100
20     100
95     100
90     100
85     100
80     100
65     100
60     100
55     100
50     100
45     100
40     100
35     100
30     100
25     100
100    100
75      99
Name: num_nodes, dtype: int64

In [520]:
results['name'] = results['name'].apply(lambda x: x.split("/")[-1])

In [521]:
z = results.set_index("name").join(processed_tf.transpose())

In [279]:
# for num_nodes, count in results['num_nodes'].value_counts().items():
#     if count > 100:
#         print(num_nodes, count - 100)
#         z = z.drop(z.loc[z['num_nodes'] == num_nodes].sample(count - 100).index)

In [280]:
z.to_json('./random_tf_con.json')

# PT Processing

In [588]:
pt_results = pd.read_json('./random_pt_con.json')

In [589]:
pt_results.loc[pt_results['error'] == 1]

,error,error_des,mad,ml1,ml2,time,num_nodes,min_dist,max_dist,mean_dist,std_dev,med_dist


In [590]:
for i, row in enumerate(pt_results.loc[pt_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

In [591]:
pt_results.loc[pt_results['error'] == 0].shape

(1792, 12)

In [592]:
pt_results.loc[pt_results['mean_dist'].notna()]

,error,error_des,mad,ml1,ml2,time,num_nodes,min_dist,max_dist,mean_dist,std_dev,med_dist
15_3490005371_pt,0,{},0,0,0,0.045101,15,{'mad': 0.0},{'mad': 5.9600000000000004e-08},{'mad': 7.7e-09},{'mad': 1.9e-08},{'mad': 0.0}
15_1830730569_pt,0,{},0,0,0,0.038187,15,{'mad': 77.9517822266},{'mad': 77.9517822266},{'mad': 77.9517822266},{'mad': 0.0},{'mad': 77.9517822266}
15_594356564_pt,0,{},0,0,0,0.028006,15,{'mad': 0.0},{'mad': 1.9073000000000001e-06},{'mad': 2.5270000000000003e-07},{'mad': 6.431e-07},{'mad': 0.0}
15_3868237654_pt,0,{},0,0,0,0.052564,15,{'mad': 4.7680000000000003e-07},{'mad': 7.6294e-06},{'mad': 2.7370000000000002e-06},{'mad': 2.2103000000000003e-06},{'mad': 1.9073000000000001e-06}
15_3294824347_pt,0,{},0,0,0,0.051622,15,{'mad': 0.0},{'mad': 2.2888200000000002e-05},{'mad': 3.7074000000000003e-06},{'mad': 5.2242e-06},{'mad': 1.1920000000000001e-07}
...,...,...,...,...,...,...,...,...,...,...,...,...
100_2038814902_pt,0,{},0,0,0,0.302235,100,{'mad': 5.4620571136},{'mad': 9.1137537956},{'mad': 6.8309248781},{'mad': 0.9517348178},{'mad': 6.7537560463}
100_723451920_pt,0,{},0,0,0,0.332820,100,{'mad': 8.8073027134},{'mad': 33024.0},{'mad': 1000.9915888047},{'mad': 3568.2635963378},{'mad': 10.3316793442}
100_2738858578_pt,0,{},0,0,0,0.346879,100,{'mad': 0.00024414060000000002},{'mad': 0.03125},{'mad': 0.0032031250000000002},{'mad': 0.0065042436},{'mad': 0.0009765625}
100_2322025169_pt,0,{},0,0,0,0.323949,100,{'mad': 4.3915224075},{'mad': 8.015474319500001},{'mad': 5.9995952702},{'mad': 0.5338748124},{'mad': 5.9650502205}


In [593]:
diff_tested_pt = pt_results.loc[(pt_results['error'] == 0) & (pt_results['mean_dist'].notna())]

In [594]:
# processed_table = pd.DataFrame(
#     columns=pd.MultiIndex.from_product(
#         [
#             pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
#             [
#                 "l1",
#                 "l2",
#                 "mad",
#             ],
#         ]
#     ),
#     index=diff_tested_pt.index,
# )


processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_pt.index,
)


In [595]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_3490005371_pt,NaN,NaN,NaN,NaN,NaN
15_1830730569_pt,NaN,NaN,NaN,NaN,NaN
15_594356564_pt,NaN,NaN,NaN,NaN,NaN
15_3868237654_pt,NaN,NaN,NaN,NaN,NaN
15_3294824347_pt,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
100_2038814902_pt,NaN,NaN,NaN,NaN,NaN
100_723451920_pt,NaN,NaN,NaN,NaN,NaN
100_2738858578_pt,NaN,NaN,NaN,NaN,NaN


In [596]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = pt_results.loc[label, col[0]][col[1]]

In [597]:
mad_median_pt_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_pt_dist.loc[mad_median_pt_dist ['mad'] > 1e-3]

,mad
15_1830730569_pt,77.951782
15_2701350503_pt,5.415833
15_3707774021_pt,31.257603
15_2664701379_pt,9.655439
15_2859127604_pt,0.265026
...,...
100_4247939447_pt,15001.112305
100_2038814902_pt,6.753756
100_723451920_pt,10.331679
100_2322025169_pt,5.96505


In [598]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'].isna()].shape

(9, 1)

In [599]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'] <= 1e-3]

,mad
15_3490005371_pt,0.0
15_594356564_pt,0.0
15_3868237654_pt,0.000002
15_3294824347_pt,0.0
15_2022779411_pt,0.0
...,...
100_670375608_pt,0.000031
100_921640703_pt,0.000031
100_2780906194_pt,0.000002
100_3187987303_pt,0.0


In [89]:
mad_median_pt_dist[['num_nodes']].value_counts().sort_index()

num_nodes
15           100
20           100
25           100
30           100
35           100
40           100
45            99
50           100
55            99
60           100
65           100
70            99
75            98
80            99
85            99
90            97
95           102
100           98
dtype: int64

# TF Processing

In [558]:
tf_results = pd.read_json('./random_tf_con.json')

In [559]:
tf_results.shape

(1800, 12)

In [560]:
tf_results.loc[tf_results['error'] == 0].shape

(1800, 12)

In [561]:
diff_tested_tf = tf_results.loc[(tf_results['error'] == 0) & (tf_results['mean_dist'].notna())]

In [562]:
# processed_table = pd.DataFrame(
#     columns=pd.MultiIndex.from_product(
#         [
#             tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
#             [
#                 "l1",
#                 "l2",
#                 "mad",
#             ],
#         ]
#     ),
#     index=diff_tested_tf.index,
# )
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                # "l1",
                # "l2",
                "mad",
            ],
        ]
    ),
    index=diff_tested_tf.index,
)

In [568]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = tf_results.loc[label, col[0]][col[1]]

In [569]:
processed_table.shape

(813, 5)

In [564]:
mad_median_tf_dist = processed_table['med_dist'][['mad']]
mad_median_tf_dist = mad_median_tf_dist.join(tf_results[['num_nodes']])
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] <= 1e-3].shape

(763, 2)

In [565]:
mad_median_tf_dist.loc[mad_median_tf_dist['mad'].isna()].shape

(19, 2)

In [566]:
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] > 1e-3].shape

(31, 2)

In [368]:
mad_median_tf_dist[['num_nodes']].value_counts().sort_index()

num_nodes
15           71
20           66
25           63
30           62
35           56
40           47
45           57
50           49
55           43
60           43
65           34
70           45
75           27
80           30
85           39
90           25
95           30
100          26
dtype: int64

In [8]:
x = pd.read_json('random_pt_old.json')
for i, val in x.loc[x['error'] == 1].iterrows():
    print(val['error_des'])

Traceback (most recent call last):
  File "/Users/jajal/research/nnsmith_testing/models/torch/__init__.py", line 44, in export_onnx
    torch.onnx.export(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 504, in export
    _export(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1529, in _export
    graph, params_dict, torch_out = _model_to_graph(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1115, in _model_to_graph
    graph = _optimize_graph(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 663, in _optimize_graph
    graph = _C._jit_pass_onnx(graph, operator_export_type)
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1899, in _run_symbolic_function
    return symbolic_fn(graph_context, *inputs, **attrs)
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onn

# PT Processing Con

In [ ]:
pt_results = pd.read_json('./random_pt_con.json')

# Conv Errors

In [22]:
pt_results.loc[pt_results['error'] == 1].shape[0]

(0, 12)

In [ ]:
for i, row in enumerate(pt_results.loc[pt_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

In [ ]:
pt_results.loc[pt_results['error'] == 0].shape

(1792, 12)

In [ ]:
pt_results.loc[pt_results['mean_dist'].notna()]

,error,error_des,mad,ml1,ml2,time,num_nodes,min_dist,max_dist,mean_dist,std_dev,med_dist
15_3490005371_pt,0,{},0,0,0,0.045101,15,{'mad': 0.0},{'mad': 5.9600000000000004e-08},{'mad': 7.7e-09},{'mad': 1.9e-08},{'mad': 0.0}
15_1830730569_pt,0,{},0,0,0,0.038187,15,{'mad': 77.9517822266},{'mad': 77.9517822266},{'mad': 77.9517822266},{'mad': 0.0},{'mad': 77.9517822266}
15_594356564_pt,0,{},0,0,0,0.028006,15,{'mad': 0.0},{'mad': 1.9073000000000001e-06},{'mad': 2.5270000000000003e-07},{'mad': 6.431e-07},{'mad': 0.0}
15_3868237654_pt,0,{},0,0,0,0.052564,15,{'mad': 4.7680000000000003e-07},{'mad': 7.6294e-06},{'mad': 2.7370000000000002e-06},{'mad': 2.2103000000000003e-06},{'mad': 1.9073000000000001e-06}
15_3294824347_pt,0,{},0,0,0,0.051622,15,{'mad': 0.0},{'mad': 2.2888200000000002e-05},{'mad': 3.7074000000000003e-06},{'mad': 5.2242e-06},{'mad': 1.1920000000000001e-07}
...,...,...,...,...,...,...,...,...,...,...,...,...
100_2038814902_pt,0,{},0,0,0,0.302235,100,{'mad': 5.4620571136},{'mad': 9.1137537956},{'mad': 6.8309248781},{'mad': 0.9517348178},{'mad': 6.7537560463}
100_723451920_pt,0,{},0,0,0,0.332820,100,{'mad': 8.8073027134},{'mad': 33024.0},{'mad': 1000.9915888047},{'mad': 3568.2635963378},{'mad': 10.3316793442}
100_2738858578_pt,0,{},0,0,0,0.346879,100,{'mad': 0.00024414060000000002},{'mad': 0.03125},{'mad': 0.0032031250000000002},{'mad': 0.0065042436},{'mad': 0.0009765625}
100_2322025169_pt,0,{},0,0,0,0.323949,100,{'mad': 4.3915224075},{'mad': 8.015474319500001},{'mad': 5.9995952702},{'mad': 0.5338748124},{'mad': 5.9650502205}


In [ ]:
diff_tested_pt = pt_results.loc[(pt_results['error'] == 0) & (pt_results['mean_dist'].notna())]

In [ ]:
# processed_table = pd.DataFrame(
#     columns=pd.MultiIndex.from_product(
#         [
#             pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
#             [
#                 "l1",
#                 "l2",
#                 "mad",
#             ],
#         ]
#     ),
#     index=diff_tested_pt.index,
# )


processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_pt.index,
)


In [ ]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_3490005371_pt,NaN,NaN,NaN,NaN,NaN
15_1830730569_pt,NaN,NaN,NaN,NaN,NaN
15_594356564_pt,NaN,NaN,NaN,NaN,NaN
15_3868237654_pt,NaN,NaN,NaN,NaN,NaN
15_3294824347_pt,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
100_2038814902_pt,NaN,NaN,NaN,NaN,NaN
100_723451920_pt,NaN,NaN,NaN,NaN,NaN
100_2738858578_pt,NaN,NaN,NaN,NaN,NaN


In [ ]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = pt_results.loc[label, col[0]][col[1]]

In [ ]:
mad_median_pt_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_pt_dist.loc[mad_median_pt_dist ['mad'] > 1e-3]

,mad
15_1830730569_pt,77.951782
15_2701350503_pt,5.415833
15_3707774021_pt,31.257603
15_2664701379_pt,9.655439
15_2859127604_pt,0.265026
...,...
100_4247939447_pt,15001.112305
100_2038814902_pt,6.753756
100_723451920_pt,10.331679
100_2322025169_pt,5.96505


In [ ]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'].isna()].shape

(9, 1)

In [ ]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'] <= 1e-3]

,mad
15_3490005371_pt,0.0
15_594356564_pt,0.0
15_3868237654_pt,0.000002
15_3294824347_pt,0.0
15_2022779411_pt,0.0
...,...
100_670375608_pt,0.000031
100_921640703_pt,0.000031
100_2780906194_pt,0.000002
100_3187987303_pt,0.0


In [ ]:
mad_median_pt_dist[['num_nodes']].value_counts().sort_index()

num_nodes
15           100
20           100
25           100
30           100
35           100
40           100
45            99
50           100
55            99
60           100
65           100
70            99
75            98
80            99
85            99
90            97
95           102
100           98
dtype: int64

# TF Processing

In [ ]:
tf_results = pd.read_json('./random_tf_con.json')

In [ ]:
tf_results.shape

(1800, 12)

In [ ]:
tf_results.loc[tf_results['error'] == 0].shape

(1800, 12)

In [ ]:
diff_tested_tf = tf_results.loc[(tf_results['error'] == 0) & (tf_results['mean_dist'].notna())]

In [ ]:
# processed_table = pd.DataFrame(
#     columns=pd.MultiIndex.from_product(
#         [
#             tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
#             [
#                 "l1",
#                 "l2",
#                 "mad",
#             ],
#         ]
#     ),
#     index=diff_tested_tf.index,
# )
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                # "l1",
                # "l2",
                "mad",
            ],
        ]
    ),
    index=diff_tested_tf.index,
)

In [ ]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = tf_results.loc[label, col[0]][col[1]]

In [ ]:
processed_table.shape

(813, 5)

In [ ]:
mad_median_tf_dist = processed_table['med_dist'][['mad']]
mad_median_tf_dist = mad_median_tf_dist.join(tf_results[['num_nodes']])
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] <= 1e-3].shape

(763, 2)

In [ ]:
mad_median_tf_dist.loc[mad_median_tf_dist['mad'].isna()].shape

(19, 2)

In [ ]:
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] > 1e-3].shape

(31, 2)

In [ ]:
mad_median_tf_dist[['num_nodes']].value_counts().sort_index()

num_nodes
15           71
20           66
25           63
30           62
35           56
40           47
45           57
50           49
55           43
60           43
65           34
70           45
75           27
80           30
85           39
90           25
95           30
100          26
dtype: int64

In [ ]:
x = pd.read_json('random_pt_old.json')
for i, val in x.loc[x['error'] == 1].iterrows():
    print(val['error_des'])

Traceback (most recent call last):
  File "/Users/jajal/research/nnsmith_testing/models/torch/__init__.py", line 44, in export_onnx
    torch.onnx.export(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 504, in export
    _export(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1529, in _export
    graph, params_dict, torch_out = _model_to_graph(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1115, in _model_to_graph
    graph = _optimize_graph(
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 663, in _optimize_graph
    graph = _C._jit_pass_onnx(graph, operator_export_type)
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onnx/utils.py", line 1899, in _run_symbolic_function
    return symbolic_fn(graph_context, *inputs, **attrs)
  File "/Users/jajal/opt/anaconda3/lib/python3.9/site-packages/torch/onn